# অগ্রাধিকার সদস্য মিডলওয়্যার সহ হোটেল বুকিং

এই নোটবুকটি মাইক্রোসফ্ট এজেন্ট ফ্রেমওয়ার্ক ব্যবহার করে **ফাংশন-ভিত্তিক মিডলওয়্যার** প্রদর্শন করে। আমরা শর্তসাপেক্ষ কর্মপ্রবাহ উদাহরণের উপর ভিত্তি করে একটি মিডলওয়্যার স্তর যোগ করেছি যা অগ্রাধিকার সদস্যদের বিশেষ সুবিধা দেয়।

## আপনি যা শিখবেন:
1. **ফাংশন-ভিত্তিক মিডলওয়্যার**: ফাংশন ফলাফল গ্রহণ এবং সংশোধন করা
2. **কন্টেক্সট অ্যাক্সেস**: কার্যকরির পরে `context.result` পড়া এবং পরিবর্তন করা
3. **ব্যবসায়িক লজিক বাস্তবায়ন**: অগ্রাধিকার সদস্য সুবিধাদি
4. **ফলাফল ওভাররাইড**: ব্যবহারকারীর স্থিতি অনুযায়ী ফাংশনের ফলাফল পরিবর্তন
5. **একই কর্মপ্রবাহ, বিভিন্ন ফলাফল**: মিডলওয়্যার দ্বারা চালিত আচরণ পরিবর্তন

## মিডলওয়্যার সহ কর্মপ্রবাহ স্থাপত্য:

```
User Input: "I want to book a hotel in Paris"
                    ↓
        [availability_agent]
        - Calls hotel_booking tool
        - 🌟 priority_check middleware intercepts
        - Checks user membership status
        - IF priority + no rooms → Override to available!
        - Returns BookingCheckResult
                    ↓
        Conditional Routing
           /                    \
    [has_availability]    [no_availability]
          ↓                      ↓
    [booking_agent]        [alternative_agent]
    (Priority override!)   (Regular users)
          ↓                      ↓
       [display_result executor]
```

## শর্তসাপেক্ষ কর্মপ্রবাহ থেকে মূল পার্থক্য:

**মিডলওয়্যার ছাড়া** (14-conditional-workflow.ipynb):
- প্যারিসে কোন রুম নেই → alternative_agent এ রুট করা

**মিডলওয়্যার সহ** (এই নোটবুক):
- নিয়মিত ব্যবহারকারী + প্যারিস → কোন রুম নেই → alternative_agent এ রুট করা
- অগ্রাধিকার ব্যবহারকারী + প্যারিস → 🌟 মিডলওয়্যার ওভাররাইড! → উপলব্ধ → booking_agent এ রুট করা

## পূর্বশর্ত:
- মাইক্রোসফ্ট এজেন্ট ফ্রেমওয়ার্ক ইনস্টল করা আছে
- শর্তসাপেক্ষ কর্মপ্রবাহের ধারণা আছে (দেখুন 14-conditional-workflow.ipynb)
- GitHub টোকেন অথবা OpenAI API চাবি
- মিডলওয়্যার প্যাটার্ন সম্পর্কে মৌলিক ধারণা


In [1]:
import asyncio
import json
import os
from collections.abc import Awaitable, Callable
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    FunctionInvocationContext,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Step 1: কাঠামোবদ্ধ আউটপুটের জন্য Pydantic মডেলগুলি সংজ্ঞায়িত করুন

এই মডেলগুলি সেই **স্কিমা** সংজ্ঞায়িত করে যা এজেন্টরা ফেরত দেবে। আমরা `priority_override` ফিল্ডটি যুক্ত করেছি যা ট্র্যাক করে কখন মিডলওয়্যার অ্যাভেলেবিলিটি রেজাল্ট পরিবর্তন করে।


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str
    priority_override: bool = False  # 🆕 NEW! Tracks if middleware overrode the result


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check with priority_override)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check with priority_override)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Step 2: অগ্রাধিকার সদস্যের ডাটাবেস নির্ধারণ

এই ডেমোর জন্য, আমরা একটি অগ্রাধিকার সদস্যপদ ডাটাবেস সিমুলেট করব। প্রোডাকশনে, এটি একটি বাস্তব ডাটাবেস বা API-কে কোয়েরি করবে।

**অগ্রাধিকার সদস্যরা:**
- `alice@example.com` - ভিআইপি সদস্য
- `bob@example.com` - প্রিমিয়াম সদস্য  
- `priority_user` - টেস্ট অ্যাকাউন্ট


In [3]:
# Simulated priority members database
PRIORITY_MEMBERS = {
    "alice@example.com",
    "bob@example.com",
    "priority_user",
}

# Global variable to track current user (in real app, use proper session management)
current_user_id = "regular_user"  # Default: regular user


def set_user(user_id: str):
    """Set the current user for the session."""
    global current_user_id
    current_user_id = user_id
    is_priority = user_id in PRIORITY_MEMBERS
    status = "🌟 PRIORITY MEMBER" if is_priority else "👤 Regular User"

    display(
        HTML(f"""
        <div style='padding: 15px; background: {"linear-gradient(135deg, #FFD700 0%, #FFA500 100%)" if is_priority else "#e3f2fd"}; 
                    border-left: 4px solid {"#FF6B35" if is_priority else "#2196f3"}; border-radius: 4px; margin: 10px 0;'>
            <strong>👤 Current User Set:</strong> {user_id}<br>
            <strong>Status:</strong> {status}
        </div>
    """)
    )


print("✅ Priority members database created")
print(f"   Priority members: {len(PRIORITY_MEMBERS)} users")

✅ Priority members database created
   Priority members: 3 users


## ধাপ ৩: হোটেল বুকিং টুল তৈরি করুন

শর্তসাপেক্ষ ওয়ার্কফ্লো এর মতোই, তবে এবার এটি মিডিলওয়্যার দ্বারা ইন্টারসেপ্ট করা হবে!


In [4]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Step 4: 🌟 প্রায়োরিটি চেক মিডলওয়্যার তৈরি করুন (মূল ফিচার!)

এই নোটবুকের এটি **মূল কার্যকারিতা**। মিডলওয়্যারটি:

1. **ইন্টারসেপ্ট** করে hotel_booking ফাংশন কল
2. `next(context)` কল করে ফাংশনটি স্বাভাবিকভাবেই **কার্যকর** করে
3. `context.result`-এ ফলাফল **পর্যালোচনা** করে
4. যদি ব্যবহারকারী প্রায়োরিটি হয় এবং কোনও কাঠামো না থাকে, তবে ফলাফল **ওভাররাইড** করে
5. পরিবর্তিত ফলাফলটি এজেন্টের কাছে **ফিরিয়ে দেয়**

**মূল প্যাটার্ন:**
```python
async def my_middleware(context, next):
    await next(context)  # ফাংশন সম্পাদন করুন
    # এখন context.result ফাংশনের আউটপুট ধারণ করে
    if some_condition:
        context.result = new_value  # ওভাররাইড!
```


In [5]:
async def priority_check_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """
    Function middleware that overrides hotel_booking results for priority members.
    
    Workflow:
    1. Let the function execute normally
    2. Check if user is a priority member
    3. If priority + no availability → Override to make rooms available!
    4. Agent will then route to booking path instead of alternative path
    """
    function_name = context.function.name

    display(
        HTML(f"""
        <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Middleware:</strong> Intercepting {function_name}...
        </div>
    """)
    )

    # Execute the original function
    await next(context)

    # Now inspect and potentially modify the result
    if context.result and function_name == "hotel_booking":
        result_data = json.loads(context.result)
        destination = result_data.get("destination", "")
        has_availability = result_data.get("has_availability", False)

        # Check if user is priority member
        is_priority = current_user_id in PRIORITY_MEMBERS

        # Override logic: Priority member + no availability → Make available!
        if is_priority and not has_availability:
            display(
                HTML(f"""
                <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); 
                            border-radius: 8px; margin: 10px 0; box-shadow: 0 4px 12px rgba(255,165,0,0.4);'>
                    <h3 style='margin: 0 0 10px 0; color: #333;'>🌟 PRIORITY OVERRIDE ACTIVATED! 🌟</h3>
                    <p style='margin: 0; color: #555; line-height: 1.6;'>
                        <strong>User:</strong> {current_user_id}<br>
                        <strong>Status:</strong> VIP Priority Member<br>
                        <strong>Action:</strong> Overriding "No Availability" for {destination}<br>
                        <strong>Result:</strong> ✅ Rooms now available for priority booking!
                    </p>
                </div>
            """)
            )

            # Override the result!
            result_data["has_availability"] = True
            result_data["priority_override"] = True
            context.result = json.dumps(result_data)

        elif not has_availability:
            display(
                HTML(f"""
                <div style='padding: 12px; background: #ffebee; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>ℹ️ Middleware:</strong> No priority override (user: {current_user_id})
                </div>
            """)
            )


print("✅ priority_check_middleware created")
print("   - Intercepts hotel_booking function")
print("   - Overrides availability for priority members")

✅ priority_check_middleware created
   - Intercepts hotel_booking function
   - Overrides availability for priority members


## Step 5: রাউটিংয়ের জন্য শর্ত ফাংশন সংজ্ঞায়িত করা

সশর্ত কার্যপ্রবাহের মতো একই শর্ত ফাংশনগুলি - তারা রাউটিং নির্ধারণের জন্য কাঠামোবদ্ধ আউটপুট পরীক্ষা করে।


In [6]:
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available (including priority overrides!)."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        # Check if this was a priority override
        override_indicator = " 🌟" if result.priority_override else ""

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}{override_indicator}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception:
        return False


print("✅ Condition functions defined")

✅ Condition functions defined


## Step 6: কাস্টম ডিসপ্লে এক্সেকিউটর তৈরি করুন

আগের মতো একই এক্সেকিউটর - চূড়ান্ত ওয়ার্কফ্লো আউটপুট প্রদর্শন করে।


In [7]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Display the final result as workflow output."""
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created")

✅ display_result executor created


## ধাপ ৭: পরিবেশ ভেরিয়েবল লোড করুন

LLM ক্লায়েন্ট (GitHub মডেল বা OpenAI) কনফিগার করুন।


In [8]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")


## ধাপ ৮: মিডলওয়্যার সহ AI এজেন্ট তৈরি করুন

**মূল পার্থক্য:** যখন availability_agent তৈরি করা হয়, আমরা `middleware` প্যারামিটারটি প্রদান করি!

এভাবেই আমরা priority_check_middleware এজেন্টের ফাংশন কলিং পাইপলাইনে ইনজেক্ট করি।


In [9]:
# Agent 1: Check availability with tool + middleware
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), message (string), "
            "and priority_override (bool, true if priority member got special access). "
            "The message should summarize the availability status and mention if priority override occurred."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
        middleware=[priority_check_middleware],  # 🌟 MIDDLEWARE INJECTION!
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "If priority_override is true in the input, mention that they received priority member access. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - WITH priority_check_middleware 🌟</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Step 9: ওয়ার্কফ্লো তৈরি করুন

আগের মতো একই ওয়ার্কফ্লো স্ট্রাকচার - প্রাপ্যতার উপর ভিত্তি করে শর্তসাপেক্ষ রাউটিং।


In [10]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH (can be triggered by middleware override!)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing (Middleware-Aware):</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> (or 🌟 <strong>priority override</strong>) → booking_agent → display_result
        </p>
    </div>
""")
)

## Step 10: টেস্ট কেস ১ - প্যারিসে নিয়মিত ব্যবহারকারী (ওভাররাইড নয়)

একজন নিয়মিত ব্যবহারকারী প্যারিসে বুক করার চেষ্টা করে → কোন রুম নেই → alternative_agent এ রুট করা হয়


In [11]:
# Set as regular user
set_user("regular_user")

display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Regular User + Paris</h3>
        <p style='margin: 0;'><strong>Expected:</strong> No rooms → No middleware override → Alternative suggestion</p>
    </div>
""")
)

# Create request
request_regular = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_regular = await workflow.run(request_regular)
outputs_regular = events_regular.get_outputs()

# Display results
if outputs_regular:
    result_regular = AlternativeResult.model_validate_json(outputs_regular[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: #fff; border: 2px solid #ff9800; border-radius: 12px; margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #e65100;'>📊 RESULT (Regular User)</h3>
            <div style='background: #fff3e0; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0;'><strong>Middleware:</strong> No priority override (regular user)</p>
                <p style='margin: 0 0 10px 0;'><strong>Alternative:</strong> 🏨 {result_regular.alternative_destination}</p>
                <p style='margin: 0;'><strong>Reason:</strong> {result_regular.reason}</p>
            </div>
        </div>
    """)
    )

## Step 11: Test Case 2 - 🌟 প্যারিসে প্রাধান্যপ্রাপ্ত ব্যবহারকারী (Override সহ!)

একজন প্রাধান্যপ্রাপ্ত সদস্য প্যারিস বুক করার চেষ্টা করেন → প্রথমে কোন রুম নেই → 🌟 Middleware ওভাররাইড করে! → booking_agent এ রুট করা হয়

**এটাই middleware শক্তির মূল প্রদর্শনী!**


In [12]:
# Set as priority user
set_user("priority_user")

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #333;'>🧪 TEST CASE 2: 🌟 Priority User + Paris</h3>
        <p style='margin: 0; color: #555;'><strong>Expected:</strong> No rooms → 🌟 MIDDLEWARE OVERRIDE → Rooms available → Booking suggestion!</p>
    </div>
""")
)

# Create request
request_priority = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_priority = await workflow.run(request_priority)
outputs_priority = events_priority.get_outputs()

# Display results
if outputs_priority:
    result_priority = BookingConfirmation.model_validate_json(outputs_priority[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; 
                    box-shadow: 0 8px 16px rgba(255,165,0,0.4); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 RESULT (Priority Member) 🌟</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Priority Override!)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> 🌟 OVERRIDE ACTIVATED!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_priority.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_priority.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_priority.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #fff3cd; border-radius: 6px; border-left: 4px solid #FF6B35;'>
                    <strong>💡 What Just Happened:</strong><br>
                    1. hotel_booking tool returned "no availability"<br>
                    2. priority_check_middleware intercepted the result<br>
                    3. Middleware checked user status: priority_user ✅<br>
                    4. Middleware OVERRODE the result to "available"<br>
                    5. Workflow routed to booking_agent instead of alternative_agent!
                </div>
            </div>
        </div>
    """)
    )

## ধাপ ১২: টেস্ট কেস ৩ - স্টকহোমের প্রায়োরিটি ব্যবহারকারী (ইতিমধ্যে উপলব্ধ)

প্রায়োরিটি ব্যবহারকারী স্টকহোম চেষ্টা করে → কক্ষ উপলব্ধ → ওভাররাইডের দরকার নেই → booking_agent এ রুট করে

এটি দেখায় যে মिडলওয়্যার শুধুমাত্র প্রয়োজনে কাজ করে!


In [13]:
# Priority user is still set from previous test

display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 3: Priority User + Stockholm</h3>
        <p style='margin: 0;'><strong>Expected:</strong> Rooms available → No override needed → Booking suggestion</p>
    </div>
""")
)

# Create request
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 RESULT (Priority User - No Override Needed)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Natural)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> No override needed</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #e8f5e9; border-radius: 6px; border-left: 4px solid #4caf50;'>
                    <strong>💡 Middleware Behavior:</strong><br>
                    • hotel_booking returned "available" naturally<br>
                    • Middleware saw available = true → No override needed<br>
                    • Workflow proceeded normally to booking_agent
                </div>
            </div>
        </div>
    """)
    )

## মূল বিষয়সমূহ এবং মাইডলওয়্যার ধারণা

### ✅ আপনি যা শিখেছেন:

#### **১. ফাংশন-ভিত্তিক মাইডলওয়্যার প্যাটার্ন**

মাইডলওয়্যার একটি সাধারণ অ্যাসিঙ্ক ফাংশন ব্যবহার করে ফাংশন কলগুলোকে আটকায়:

```python
async def my_middleware(
    context: FunctionInvocationContext,
    next: Callable,
) -> None:
    # ফাংশন কার্যনির্বাহের আগে
    print("Intercepting...")
    
    # ফাংশন কার্যনির্বাহ করুন
    await next(context)
    
    # ফাংশন কার্যনির্বাহের পরে - ফলাফল পরিদর্শন করুন
    if context.result:
        # প্রয়োজনে ফলাফল পরিবর্তন করুন
        context.result = modified_value
```

#### **২. প্রেক্ষাপট অ্যাক্সেস এবং ফলাফল ওভাররাইড**

- `context.function` - কল করা ফাংশনে প্রবেশাধিকার
- `context.arguments` - ফাংশনের আর্গুমেন্ট পড়া
- `context.kwargs` - অতিরিক্ত প্যারামিটার অ্যাক্সেস করা
- `await next(context)` - ফাংশন কার্যকর করা
- `context.result` - ফাংশনের আউটপুট পড়া/পরিবর্তন করা

#### **৩. ব্যবসায়িক লজিক বাস্তবায়ন**

আমাদের মাইডলওয়্যার অগ্রাধিকার সদস্য সুবিধাসমূহ বাস্তবায়ন করে:
- **নিয়মিত ব্যবহারকারী**: কোনও পরিবর্তন নয়, সাধারণ কার্যপ্রবাহ
- **অগ্রাধিকার ব্যবহারকারী**: "কোনো উপলভ্যতা নেই" → "উপলভ্য" ওভাররাইড করে
- **শর্তাধীন লজিক**: শুধুমাত্র প্রয়োজন অনুযায়ী ওভাররাইড করে

#### **৪. একই কার্যপ্রবাহ, পৃথক ফলাফল**

মাইডলওয়্যারের ক্ষমতা:
- ✅ কার্যপ্রবাহ কাঠামোতে কোনও পরিবর্তন নেই
- ✅ টুল ফাংশনে কোনও পরিবর্তন নেই
- ✅ শর্তাধীন রুটিং লজিকে কোনও পরিবর্তন নেই
- ✅ শুধু মাইডলওয়্যার → ভিন্ন আচরণ!

### 🚀 বাস্তব-জগতের ব্যবহারসমূহ:

১. **ভিআইপি/প্রিমিয়াম ফিচার**
   - প্রিমিয়াম ব্যবহারকারীদের জন্য রেট লিমিট ওভাররাইড করা
   - সম্পদের অগ্রাধিকার অ্যাক্সেস প্রদান করা
   - ডায়নামিক প্রিমিয়াম ফিচার আনলক করা

২. **এ/বি টেস্টিং**
   - ব্যবহারকারীদের ভিন্ন ইমপ্লিমেন্টেশনে রুট করা
   - নির্দিষ্ট ব্যবহারকারীর সঙ্গে নতুন ফিচার পরীক্ষা করা
   - ধাপে ধাপে ফিচার রোলআউট

৩. **নিরাপত্তা ও সম্মতি**
   - ফাংশন কল অডিট করা
   - সংবেদনশীল অপারেশন ব্লক করা
   - ব্যবসায়িক নিয়ম জোরদার করা

৪. **পারফরম্যান্স অপটিমাইজেশন**
   - নির্দিষ্ট ব্যবহারকারীর জন্য ফলাফল ক্যাশ করা
   - সম্ভব হলে ব্যয়বহুল অপারেশন বাদ দেওয়া
   - ডায়নামিক রিসোর্স বরাদ্দ

৫. **ত্রুটি পরিচালনা এবং পুনরায় চেষ্টা**
   - ত্রুটি ধরা ও সুষ্ঠুভাবে পরিচালনা করা
   - পুনরায় চেষ্টা করার লজিক বাস্তবায়ন করা
   - বিকল্প ইমপ্লিমেন্টেশনে ফ্যালব্যাক করা

৬. **লগিং এবং মনিটরিং**
   - ফাংশন কার্যকর করার সময় ট্র্যাক করা
   - প্যারামিটার এবং ফলাফল লগ করা
   - ব্যবহার ধরণ পর্যবেক্ষণ করা

### 🔑 ডেকোরেটরের থেকে মূল পার্থক্য:

| বৈশিষ্ট্য | ডেকোরেটর | মাইডলওয়্যার |
|---------|-----------|------------|
| **পরিধি** | একক ফাংশন | এজেন্টের সকল ফাংশন |
| **নমনীয়তা** | সংজ্ঞায়িত হওয়ার সময় নির্দিষ্ট | রানটাইমে ডায়নামিক |
| **প্রেক্ষাপট** | সীমাবদ্ধ | পূর্ণ এজেন্ট প্রেক্ষাপট |
| **সংরচনা** | একাধিক ডেকোরেটর | মাইডলওয়্যার পাইলাইন |
| **এজেন্ট-সচেতন** | না | হ্যাঁ (এজেন্ট স্টেটে প্রবেশাধিকার) |

### 📚 কখন মাইডলওয়্যার ব্যবহার করবেন:

✅ **মাইডলওয়্যার ব্যবহার করুন যখন:**
- ব্যবহারকারী/সেশন স্টেট অনুযায়ী আচরণ পরিবর্তন করতে চান
- একাধিক ফাংশনে লজিক প্রয়োগ করতে চান
- এজেন্ট-স্তরের প্রেক্ষাপটে প্রবেশাধিকার প্রয়োজন
- ক্রস-কাটিং বিষয়াদি (লগিং, অথ, ইত্যাদি) বাস্তবায়ন করছেন

❌ **মাইডলওয়্যার ব্যবহার করবেন না যখন:**
- সহজ ইনপুট যাচাই (Pydantic ব্যবহার করুন)
- ফাংশন নির্দিষ্ট লজিক (ফাংশনের মধ্যে রাখুন)
- একবারের পরিবর্তন (সরাসরি ফাংশন পরিবর্তন করুন)

### 🎓 উন্নত প্যাটার্নসমূহ:

```python
# একাধিক মিডলওয়্যার (কার্যকরী ক্রম গুরুত্বপূর্ণ!)
middleware=[
    logging_middleware,      # প্রথমে লগস
    auth_middleware,         # তারপর অথ চেক করে
    cache_middleware,        # এরপর ক্যাশ চেক করে
    rate_limit_middleware,   # তারপর রেট লিমিট করে
    priority_check_middleware  # অবশেষে প্রাধান্য চেক
]

# শর্তাধীন মিডলওয়্যার কার্যকরী
async def conditional_middleware(context, next):
    if should_execute(context):
        await next(context)
        # ফলাফল পরিবর্তন করুন
    else:
        # সম্পূর্ণ কার্যকরী এড়িয়ে যান
        context.result = cached_value
```

### 🔗 সংশ্লিষ্ট ধারণাসমূহ:

- **এজেন্ট মাইডলওয়্যার**: agent.run() কল আটকায়
- **ফাংশন মাইডলওয়্যার**: টুল ফাংশন কল আটকায় (আমরা যেটা ব্যবহার করেছি!)
- **মাইডলওয়্যার পাইলাইন**: ক্রমান্বয়ে কার্যকর হওয়ার মাইডলওয়্যার চেইন
- **প্রেক্ষাপট প্রসার**: মাইডলওয়্যার চেইনের মাধ্যমে স্টেট প্রদান করা


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ সেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনূদিত হয়েছে। আমরা যথাসাধ্য সঠিকতার জন্য চেষ্টা করি, তবে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসামঞ্জস্য থাকতে পারে। মূল ভাষায় থাকা নথিটিই কর্তৃত্বপূর্ণ উৎস হিসেবে গণ্য করা উচিত। গুরত্বপূর্ণ তথ্যের জন্য পেশাদার মানুষের অনুবাদ গ্রহণ করার পরামর্শ দেওয়া হয়। এই অনুবাদের ব্যবহারে কোনও ভুল বোধ বা ভুল ব্যাখ্যার জন্য আমরা দায়ী থাকব না।
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
